# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 5 2022 1:17PM,243567,20,8109934-BO,"ACI Healthcare USA, Inc.",Released
1,Aug 5 2022 1:17PM,243567,20,8138394,"ACI Healthcare USA, Inc.",Released
2,Aug 5 2022 12:50PM,243566,10,MSP212867,"Methapharm, Inc.",Released
3,Aug 5 2022 12:50PM,243566,10,MSP212889,"Methapharm, Inc.",Released
4,Aug 5 2022 12:50PM,243566,10,MSP212883,"Methapharm, Inc.",Released
5,Aug 5 2022 12:50PM,243566,10,MSP212896,"Methapharm, Inc.",Released
6,Aug 5 2022 12:50PM,243565,10,MSP212882,"Methapharm, Inc.",Released
7,Aug 5 2022 12:50PM,243565,10,MSP212884,"Methapharm, Inc.",Released
8,Aug 5 2022 12:50PM,243565,10,MSP212885,"Methapharm, Inc.",Released
9,Aug 5 2022 12:50PM,243565,10,MSP212886,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,243560,Released,1
27,243561,Released,1
28,243565,Released,18
29,243566,Released,4
30,243567,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243560,NaN,NaN,1.0
243561,NaN,NaN,1.0
243565,NaN,NaN,18.0
243566,NaN,NaN,4.0
243567,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243478,0.0,0.0,1.0
243485,5.0,2.0,0.0
243497,0.0,1.0,0.0
243508,0.0,0.0,1.0
243509,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243478,0,0,1
243485,5,2,0
243497,0,1,0
243508,0,0,1
243509,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243478,0,0,1
1,243485,5,2,0
2,243497,0,1,0
3,243508,0,0,1
4,243509,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243478,,,1
1,243485,5,2,
2,243497,,1,
3,243508,,,1
4,243509,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 5 2022 1:17PM,243567,20,"ACI Healthcare USA, Inc."
2,Aug 5 2022 12:50PM,243566,10,"Methapharm, Inc."
6,Aug 5 2022 12:50PM,243565,10,"Methapharm, Inc."
24,Aug 5 2022 12:14PM,243561,19,"GUSA Granules USA, Inc."
25,Aug 5 2022 12:09PM,243560,10,Eywa Pharma Inc.
26,Aug 5 2022 12:05PM,243559,19,"GUSA Granules USA, Inc."
27,Aug 5 2022 11:59AM,243558,19,"AdvaGen Pharma, Ltd"
31,Aug 5 2022 11:36AM,243555,19,"GUSA Granules USA, Inc."
32,Aug 5 2022 11:35AM,243556,16,Sartorius Stedim Filters Inc.
33,Aug 5 2022 11:34AM,243554,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 5 2022 1:17PM,243567,20,"ACI Healthcare USA, Inc.",,,2
1,Aug 5 2022 12:50PM,243566,10,"Methapharm, Inc.",,,4
2,Aug 5 2022 12:50PM,243565,10,"Methapharm, Inc.",,,18
3,Aug 5 2022 12:14PM,243561,19,"GUSA Granules USA, Inc.",,,1
4,Aug 5 2022 12:09PM,243560,10,Eywa Pharma Inc.,,,1
5,Aug 5 2022 12:05PM,243559,19,"GUSA Granules USA, Inc.",,,1
6,Aug 5 2022 11:59AM,243558,19,"AdvaGen Pharma, Ltd",,1,3
7,Aug 5 2022 11:36AM,243555,19,"GUSA Granules USA, Inc.",,,1
8,Aug 5 2022 11:35AM,243556,16,Sartorius Stedim Filters Inc.,,,1
9,Aug 5 2022 11:34AM,243554,16,Sartorius Stedim Filters Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 1:17PM,243567,20,"ACI Healthcare USA, Inc.",2,,
1,Aug 5 2022 12:50PM,243566,10,"Methapharm, Inc.",4,,
2,Aug 5 2022 12:50PM,243565,10,"Methapharm, Inc.",18,,
3,Aug 5 2022 12:14PM,243561,19,"GUSA Granules USA, Inc.",1,,
4,Aug 5 2022 12:09PM,243560,10,Eywa Pharma Inc.,1,,
5,Aug 5 2022 12:05PM,243559,19,"GUSA Granules USA, Inc.",1,,
6,Aug 5 2022 11:59AM,243558,19,"AdvaGen Pharma, Ltd",3,1,
7,Aug 5 2022 11:36AM,243555,19,"GUSA Granules USA, Inc.",1,,
8,Aug 5 2022 11:35AM,243556,16,Sartorius Stedim Filters Inc.,1,,
9,Aug 5 2022 11:34AM,243554,16,Sartorius Stedim Filters Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 1:17PM,243567,20,"ACI Healthcare USA, Inc.",2,,
1,Aug 5 2022 12:50PM,243566,10,"Methapharm, Inc.",4,,
2,Aug 5 2022 12:50PM,243565,10,"Methapharm, Inc.",18,,
3,Aug 5 2022 12:14PM,243561,19,"GUSA Granules USA, Inc.",1,,
4,Aug 5 2022 12:09PM,243560,10,Eywa Pharma Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 1:17PM,243567,20,"ACI Healthcare USA, Inc.",2.0,NaN,NaN
1,Aug 5 2022 12:50PM,243566,10,"Methapharm, Inc.",4.0,NaN,NaN
2,Aug 5 2022 12:50PM,243565,10,"Methapharm, Inc.",18.0,NaN,NaN
3,Aug 5 2022 12:14PM,243561,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Aug 5 2022 12:09PM,243560,10,Eywa Pharma Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 1:17PM,243567,20,"ACI Healthcare USA, Inc.",2.0,0.0,0.0
1,Aug 5 2022 12:50PM,243566,10,"Methapharm, Inc.",4.0,0.0,0.0
2,Aug 5 2022 12:50PM,243565,10,"Methapharm, Inc.",18.0,0.0,0.0
3,Aug 5 2022 12:14PM,243561,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Aug 5 2022 12:09PM,243560,10,Eywa Pharma Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1461209,46.0,9.0,0.0
12,243497,0.0,1.0,0.0
15,243509,1.0,0.0,0.0
16,2191827,7.0,2.0,0.0
19,1217781,7.0,10.0,0.0
20,974133,35.0,21.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1461209,46.0,9.0,0.0
1,12,243497,0.0,1.0,0.0
2,15,243509,1.0,0.0,0.0
3,16,2191827,7.0,2.0,0.0
4,19,1217781,7.0,10.0,0.0
5,20,974133,35.0,21.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,9.0,0.0
1,12,0.0,1.0,0.0
2,15,1.0,0.0,0.0
3,16,7.0,2.0,0.0
4,19,7.0,10.0,0.0
5,20,35.0,21.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,12,Released,0.0
2,15,Released,1.0
3,16,Released,7.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,5.0
Executing,9.0,1.0,0.0,2.0,10.0,21.0
Released,46.0,0.0,1.0,7.0,7.0,35.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0
1,Executing,9.0,1.0,0.0,2.0,10.0,21.0
2,Released,46.0,0.0,1.0,7.0,7.0,35.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0
1,Executing,9.0,1.0,0.0,2.0,10.0,21.0
2,Released,46.0,0.0,1.0,7.0,7.0,35.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()